In [ ]:
import plotly.graph_objects as go
import numpy as np
import uwacan

In [ ]:
wave_filepath = r"d:\2023-08-30_12-17-53.wav"
int32_timedata_filepath = r"d:\tern_island_2023-08-30_12-17-53.bin"
zero_spectrogram_path = r"d:\monday_out.npy"

In [ ]:
sensors = (
    uwacan.sensor('Colmar 1', depth=46 - 2, sensitivity=-164.09, position=(57, 37.3070, 11, 34.4888))
    & uwacan.sensor('Colmar 2', depth=46 - 12, sensitivity=-164.24, position=(57, 37.3070, 11, 34.4888))
)
recording = uwacan.recordings.MultichannelAudioInterfaceRecording(
    files=[
        uwacan.recordings.MultichannelAudioInterfaceRecording.RecordedFile(
            filepath=wave_filepath,
            start_time=uwacan._core.time_to_datetime("2023-08-30 12:17:53Z"),
            channels=[0, 1],
        )
    ],
    sensor=sensors,
)
num_channels = recording.num_channels
samplerate = recording.samplerate

In [ ]:
raw_wav_data = recording.raw_data()
(raw_wav_data * 2**31).astype("int32").tofile("written_data.bin")

In [ ]:
spectrogram = uwacan.spectral.Spectrogram(
    recording,
    hybrid_resolution=0.2,
    frame_duration=5,
    bands_per_decade=100,
    frame_overlap=0.5
)

In [ ]:
fig = spectrogram.make_figure()
fig.add_trace(spectrogram.sel(sensor='Colmar 1').plot())

In [ ]:
power_from_zero = np.load(zero_spectrogram_path)
scaled_from_zero = power_from_zero / (2**31)**2
level_from_zero = 10*np.log10(scaled_from_zero)
calibrated_from_zero = level_from_zero - sensors.sensitivity.values

In [ ]:
go.Figure(
    go.Heatmap(
        x=spectrogram.time,
        y=spectrogram.frequency,
        z=calibrated_from_zero[...,0],
        transpose=True,
        colorscale="viridis"
    ),
    dict(
        yaxis=dict(
            type="log"
        )
    )
)

In [ ]:
# np.abs(10*np.log10(spectrogram.data.data[3:105]) - calibrated_from_zero[:102]).mean()  # Old test, when the exported data did not line up with the original file
np.abs(10 * np.log10(spectrogram.data.data[:, 1:-1]) - calibrated_from_zero[:, 1:-1]).mean()  # Comparing power from zero with old implementation, except DC and Nyquist.

In [ ]:
def scaled_window(window, scaling="density", samplerate=1):
    if scaling == "density":
        power_scale = 2 / (np.sum(window**2) * samplerate)
    elif scaling == "spectrum":
        power_scale = 2 / np.sum(window)**2

    return window * power_scale**0.5


def spectrum(time_data, window=None, detrend=True, axis=0):
    """Compute the spectal power of a signal.

    Notes
    -----
    This function always operates over the first axis (axis=0) of the data.
    """
    time_data = np.moveaxis(time_data, axis, -1)

    if detrend:
        # We should never do this in place, since it will break with overlapping frames.
        time_data = time_data - time_data.mean(axis=-1, keepdims=True)

    if window is not None:
        # Again, never in place since it breaks overlapping frames.
        time_data = time_data * window

    nfft = time_data.shape[-1]
    freq_data = np.fft.rfft(time_data, axis=-1, n=nfft)
    freq_data = np.abs(freq_data) ** 2

    return np.moveaxis(freq_data, -1, axis)


def linear_to_banded(linear_spectrum, lower_edges, upper_edges, spectral_resolution):
    banded_spectrum = np.full(lower_edges.shape + linear_spectrum.shape[1:], np.nan)
    for band_idx, (lower_edge, upper_edge) in enumerate(zip(lower_edges, upper_edges)):
        lower_idx = int(np.floor(lower_edge / spectral_resolution + 0.5))
        upper_idx = int(np.ceil(upper_edge / spectral_resolution - 0.5))
        lower_idx = max(lower_idx, 0)
        upper_idx = min(upper_idx, linear_spectrum.shape[0] - 1)

        if lower_idx == upper_idx:
            banded_spectrum[band_idx] = linear_spectrum[lower_idx]
        else:
            first_weight = lower_idx + 0.5 - lower_edge / spectral_resolution
            last_weight = upper_edge / spectral_resolution - upper_idx + 0.5
            this_band = (
                linear_spectrum[lower_idx + 1 : upper_idx].sum(axis=0)
                + linear_spectrum[lower_idx] * first_weight
                + linear_spectrum[upper_idx] * last_weight
            )
            banded_spectrum[band_idx] = this_band * (spectral_resolution / (upper_edge - lower_edge))
    return banded_spectrum


def hybrid_band_edges(bands_per_decade, hybrid_resolution, nfft, samplerate, min_frequency=None, max_frequency=None):
    # linear_frequency = np.arange(nfft // 2 + 1) * samplerate / nfft
    max_frequency = max_frequency or samplerate / 2
    min_frequency = min_frequency or 0

    #if min_frequency:
    #    lower_index = int(np.ceil(min_frequency / samplerate))
    #else:
    #    lower_index = None
    #if max_frequency:
    #    upper_index = int(np.floor(max_frequency / samplerate))
    #else:
    #    upper_index = None

    # Assuming that bands per decade is given. If we're not doing banded processing, we should not use linear to banded, and we won't need to calculate the band edges.
    log_band_scaling = 10 ** (0.5 / bands_per_decade)
    if not hybrid_resolution:
        first_lin_idx = last_lin_idx = 0
        first_log_idx = np.round(bands_per_decade * np.log10(min_frequency))
    else:
        minimum_bandwidth_frequency = hybrid_resolution / (log_band_scaling - 1 / log_band_scaling)
        first_log_idx = int(np.ceil(bands_per_decade * np.log10(minimum_bandwidth_frequency)))
        last_lin_idx = int(np.floor(minimum_bandwidth_frequency / hybrid_resolution))

        while (last_lin_idx + 0.5) * hybrid_resolution > 10 ** ((first_log_idx - 0.5) / bands_per_decade):
            last_lin_idx += 1
            first_log_idx += 1

        if last_lin_idx * hybrid_resolution > max_frequency:
            last_lin_idx = int(np.floor(max_frequency / hybrid_resolution))
        first_lin_idx = int(np.ceil(min_frequency / hybrid_resolution))
    last_log_idx = round(bands_per_decade * np.log10(max_frequency))

    lin_centers = np.arange(first_lin_idx, last_lin_idx) * hybrid_resolution
    lin_lowers = lin_centers - 0.5 * hybrid_resolution
    lin_uppers = lin_centers + 0.5 * hybrid_resolution

    log_centers = 10 ** (np.arange(first_log_idx, last_log_idx + 1) / bands_per_decade)
    log_lowers = log_centers / log_band_scaling
    log_uppers = log_centers * log_band_scaling

    centers = np.concatenate([lin_centers, log_centers])
    lowers = np.concatenate([lin_lowers, log_lowers])
    uppers = np.concatenate([lin_uppers, log_uppers])

    if centers[0] < min_frequency:
        mask = centers >= min_frequency
        centers = centers[mask]
        lowers = lowers[mask]
        uppers = uppers[mask]
    if centers[-1] > max_frequency:
        mask = centers <= max_frequency
        centers = centers[mask]
        lowers = lowers[mask]
        uppers = uppers[mask]

    return centers, (lowers, uppers)


def hybrid_spectrogram(
    data,
    samplerate,
    frame_duration,
    frame_overlap,
    bands_per_decade,
    hybrid_resolution,
    frames_to_average=1,
    output=None,
    ):

    samples_per_frame = int(np.ceil(samplerate * frame_duration))
    sample_step = int(np.floor(samplerate * frame_duration * (1 - frame_overlap)))
    # sample_overlap = samples_per_frame - sample_step
    num_frames = (data.shape[0] - samples_per_frame) // sample_step + 1
    num_frames //= frames_to_average

    window = np.sin(np.linspace(0, np.pi, samples_per_frame))**2
    window = scaled_window(window, scaling="density", samplerate=samplerate)

    frequency, (lower_edges, upper_edges) = hybrid_band_edges(
        bands_per_decade=bands_per_decade,
        hybrid_resolution=hybrid_resolution,
        nfft=samples_per_frame,
        samplerate=samplerate,
    )

    output_shape = (num_frames, frequency.size) + data.shape[1:]
    if output is None:
        output = np.full(output_shape, np.nan, dtype="float32")
    elif isinstance(output, str):
        if output.endswith(".npy"):
            output = np.lib.format.open_memmap(output, mode="w+", dtype="float32", shape=output_shape)

    for frame_idx in range(num_frames):
        avg_frame = 0
        for _ in range(frames_to_average):
            frame = data[frame_idx * sample_step : frame_idx * sample_step + samples_per_frame]
            frame = spectrum(frame, window=window)
            frame = linear_to_banded(frame, lower_edges, upper_edges, samplerate / samples_per_frame)
            avg_frame += frame

        output[frame_idx] = avg_frame / frames_to_average

    if isinstance(output, np.memmap):
        output.flush()
    return output

In [ ]:
int32_timedata = np.memmap(int32_timedata_filepath, dtype="int32").reshape((-1, num_channels))

power_as_zero = hybrid_spectrogram(
    data=int32_timedata,
    samplerate=samplerate,
    frame_duration=5,
    frame_overlap=0.5,
    bands_per_decade=100,
    hybrid_resolution=0.2,
    frames_to_average=1,
)

In [ ]:
scaled_as_zero = power_as_zero / (2**31)**2
level_as_zero = 10*np.log10(scaled_as_zero)
calibrated_as_zero = level_as_zero - sensors.sensitivity.values

In [ ]:
# Average absolute level diff, excluding DC and nyquist.
# The current code on the pi zero does not properly scale DC and nyquist.
np.mean(np.abs(calibrated_as_zero[:, 1:-1] - 10*np.log10(spectrogram.data.values[:, 1:-1])))

In [ ]:
go.Figure(
    go.Heatmap(
        x=spectrogram.time,
        y=spectrogram.frequency,
        z=calibrated_as_zero[..., 1],
        transpose=True,
        colorscale="viridis",
    ),
    dict(
        yaxis=dict(
            type="log"
        )
    )
)

In [ ]:
go.Figure(
    go.Heatmap(
        x=spectrogram.time,
        y=spectrogram.frequency,
        z=calibrated_as_zero[:, :-1, 0] - 10*np.log10(spectrogram.data.values[:, :-1, 0]),
        transpose=True,
        colorscale="rdylbu",
        zmin=-0.00001, zmax=0.00001
    ),
    dict(
        # yaxis=dict(
        #     type="log"
        # )
    )
)